In [38]:
from pathlib import Path
import pickle

hitlist_file = Path(".") / "../datasets/internet_address_verfploeter_hitlist_it102w-20230125.fsdb"

ip_hitlist = {}
with open(hitlist_file, "r") as f:
    for i, row in enumerate(f.readlines()[1:]):
        row = row.split("\t")

        # get prefix from hex value
        prefix_hex = row[0]
        prefix = ["".join(x) for x in zip(*[iter(prefix_hex)]*2)]
        prefix = [int(x, 16) for x in prefix]
        prefix = ".".join(str(x) for x in prefix)

        ip_list = row[-1].strip("\n")
        ip_list = ip_list.split(",")

        if ip_list[0] != '-' : 
            ip_hitlist[prefix] = []
            for byte in ip_list:
                ip_addr = prefix.split(".")[:-1]
                ip_addr.append(str(int(byte, 16)))
                ip_addr = ".".join(ip_addr)
                ip_hitlist[prefix].append(ip_addr)
                

out_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(out_file, "wb") as f:
    pickle.dump(ip_hitlist, f)


In [2]:
import pickle
from pathlib import Path

in_file = Path(".") / "../datasets/parsed_hitlist.pickle"
with open(in_file, "rb") as f:
    ip_hitlist = pickle.load(f)

anchor_file = "../datasets/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

In [3]:
from ipaddress import IPv4Address, IPv4Network
from random import randint

for i, anchor_ip in enumerate(anchors):
    nb_ip_addresses = 3
    if i > 10: break
    prefix_anchor = anchor_ip.split(".")[:-1]
    prefix_anchor.append("0")
    prefix_anchor = ".".join(prefix_anchor)
    ip_list = []
    try:
        ip_list = ip_hitlist[prefix_anchor]
    except KeyError:
        pass

    if len(ip_list) < nb_ip_addresses:

        ip = anchor_ip.split(".")[:2]
        ip.extend(['0','0'])
        ip = ".".join(ip)
        prefix_target = IPv4Network(ip + '/' + str(24))

        ip_list.extend([str(prefix_target[randint(1,254)]) for _ in range(0,nb_ip_addresses - len(ip_list))])
        print(f"ip list for anchor {prefix_anchor}: {ip_list}")



ip list for anchor 76.26.115.0: ['76.26.115.193', '76.26.115.194', '76.26.0.214']
ip list for anchor 199.10.66.0: ['199.10.0.146', '199.10.0.191', '199.10.0.101']
